In [1]:
import os

print(os.listdir('/work/bb1203/g260190_heinrich/Dunkelflaute/Data_Scripts'))
print(os.path.exists('/work/bb1203/g260190_heinrich/Dunkelflaute/Data_Scripts'))

['calc_wind.py', 'test.ipynb', 'find_data.py', 'EUR-11_mask.nc', 'main.py', 'calc_dunkelflaute.py', 'Bash_region.sh', 'helper_functions.py', 'nukleus_files.json', 'calc_pv.py', 'Bash_main.sh', '__pycache__', 'regional.o18916722', 'config.yaml', ' __init__.py', 'regional_data.py', 'CEU-3_mask.nc']
True


In [27]:
import xarray as xr
import pandas as pd
import os
import json
import re
def get_sorted_nc_files(folder_path: str, substring=None):
    """
    Returns a sorted list of all nc_files in the folder_path.
    """
    nc_files = [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.endswith(".nc")
        and os.path.isfile(os.path.join(folder_path, f))
        and (substring is None or substring in f)
    ]
    return sorted(nc_files)

info = {}
files = get_sorted_nc_files("/work/bb1203/g260190_heinrich/Dunkelflaute/Data/Duisburg/Wind/")
for file in files:
    if "EUR-11" in file:
        continue

    with xr.open_dataset(file) as df:
        start = pd.to_datetime(df.time.values[0]).year
        end = pd.to_datetime(df.time.values[-1]).year

        base_name = os.path.basename(file)

        info[base_name] = {}
        if "historical" in file:
            info[base_name]["historical"] = {"start": int(start), "end": int(end)}
        else:
            scenario = re.search(r'GWL\dK', file)
            if scenario:
                scenario_key = scenario.group()
                info[base_name][scenario_key] = {"start": int(start), "end": int(end)}


        file_names = base_name.replace("CEU-3", "EUR-11")
        cleaned = re.sub(r'-GWL\dK', '', file_names)
        eur_11_file_split = cleaned.split("_")

        for match_file in files:
            if all(sub in match_file for sub in eur_11_file_split):
                match_base = os.path.basename(match_file)

                if match_base not in info:
                    info[match_base] = {}

                if "historical" in file:
                    info[match_base]["historical"] = {"start": int(start), "end": int(end)}
                else:
                    scenario = re.search(r'GWL\dK', file)
                    if scenario:
                        scenario_key = scenario.group()
                        info[match_base][scenario_key] = {"start": int(start), "end": int(end)}
                break


with open("json_file.json", "w") as file:
            json.dump(info, file, indent=4)
